# Open Loop Operation

An example of scanning over LED brightnesses and plotting the result in the photodiode

In [16]:
import numpy as np
import epics
from epics import PV

## Setup

### Set all six LEDs to flash in unison

Create a six-element list of LED-duration process variables

In [17]:
LEDdurs = [None]*6
for i in range(6):
    LEDdurs[i] = PV('LEDS:CH' + str(i) + ':dur.VAL')

Set all LEDs to flash for the same duration: five hundred milliseconds.

In [18]:
for LEDdur in LEDdurs:
    LEDdur.put(500e3)

### Initialize all six LEDs at their maximum brightness

Create a six-element list of LED-brightness process variables

In [19]:
LEDbrigs = [None]*6
for i in range(6):
    LEDbrigs[i] = PV('LEDS:CH' + str(i) + ':brig.VAL')

Set all LEDs to the same brightness level: 255 (out of 255)

In [20]:
for LEDbrig in LEDbrigs:
    LEDbrig.put(255)

### Ensure the shutter is open throughout LED flash

Specify that the shutter should wait **a full 1.5 seconds** to start closing after opening

In [21]:
epics.caput("SHUTTER:dur", 1.5e6)

1

Reduce the overall repetition rate of the system to once every three seconds

In [24]:
epics.caput("PULSEGEN:reprate", 1/3.0)

1

Increase the delay time on the LED trigger to be 400 milliseconds (it started at 2 milliseconds)

In [37]:
epics.caput("PULSEGEN:CH0:delay", 400.0e3)

1

## Read the photodetector value

In [38]:
epics.caget()

400000.0